In [1]:
from llama_index.core import SimpleDirectoryReader

In [2]:
loader = SimpleDirectoryReader(
    input_files = ["Lec1_CSCI567.pdf", "Slides_Exp.pdf"],
    required_exts = [".pdf"],
    recursive = True
)

docs = loader.load_data()

In [19]:
for doc in docs:
    string = doc.text
    string = string.replace('\n', ' ')
    doc.text = string

In [23]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-large-en-v1.5")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [24]:
import qdrant_client
from llama_index.core import Settings

Settings.embed_model = embed_model

client = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)

In [25]:
from qdrant_client.models import Distance, VectorParams, Batch
from llama_index.core import StorageContext
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex


unique_collection_name = "documents"
vector_store = QdrantVectorStore(client=client, collection_name = unique_collection_name)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    docs,
    storage_context=storage_context,
)

In [26]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [27]:
from llama_index.llms.ollama import Ollama

llm=Ollama(model="llama3", request_timeout=90.0, device = device)

Settings.llm = llm
query_engine = index.as_query_engine()

In [30]:
from llama_index.core import PromptTemplate

qa_prompt_tmpl_str = (
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information above I want you to think step by step to answer the query in detail and also write the refered text in (""), incase case you don't know the answer say 'I don't know!'.\n"
            "Query: {query_str}\n"
            "Answer: "
            )

qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)
query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_tmpl})

In [31]:
response = query_engine.query('What do we mean by ERM?')
print(response)

Based on the context information provided, I can attempt to answer your query step by step.

ERM stands for Empirical Risk Minimizer. Let's break it down:

* "Empirical" refers to the fact that we're dealing with a specific set of data points (S) and calculating the average loss over those data points.
* "Risk" is the expected loss or error of our predictor function f, given the input space X and output space Y. In other words, it's the expected difference between our predicted outputs and the true labels.
* "Minimizer" means that we're trying to find the optimal (or closest) function in the function class F that minimizes this risk or expected loss.

Mathematically, this is expressed as minimizing Rs(f), which is the average loss over all data points in the set S. The formula for Rs(f) is given as:

Rs(f) = (1/n) * Σ l(f(xi), yi) for i from 1 to n

where n is the number of data points, xi and yi are the input and corresponding output for each data point, and l is the loss function.

S